## Imports

In [1]:
# Imports
import os
import pandas as pd
import numpy as np

## Call Data Folder

In [2]:
# Call 'Data' folder
os.listdir("Data/")

['.ipynb_checkpoints',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'title_akas.csv.gz']

## Load Datasets

In [3]:
# Load 'basics' dataset
basics = pd.read_csv('Data/title.basics.tsv.gz',sep='\t', low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
# Load datatypes
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10064703 entries, 0 to 10064702
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 691.1+ MB


In [5]:
# Load filtered 'akas' dataset
akas = pd.read_csv('Data/title_akas.csv.gz', low_memory=False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


## Filtering/Cleaning Steps:
- Title Basics:
 - Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)
 - Replace "\N" with np.nan
 - Eliminate movies that are null for runtimeMinutes
 - Eliminate movies that are null for genre
 - keep only titleType==Movie
 - Convert the startYear column to float data type.
 - Filter the dataframe using startYear. Keep years between 2000-2021 (Including 2000 and 2021)
 - Eliminate movies that include "Documentary" in the genre (see tip below).

In [6]:
# Keep only US movies
keepers = basics['tconst'].isin(akas['titleId'])
keepers

0            True
1            True
2           False
3           False
4            True
            ...  
10064698    False
10064699    False
10064700    False
10064701    False
10064702    False
Name: tconst, Length: 10064703, dtype: bool

In [7]:
# Filter the basics table down to only include the US by using the filter akas dataframe
basics = basics[keepers]

In [8]:
# Replace "\N" with np.nan
basics.replace({'\\N':np.nan}, inplace=True)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,NaN,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,NaN,1,"Short,Sport"


In [9]:
# Eliminate movies that are null for the 'runtimeMinutes' and 'genres' columns
basics = basics.dropna(subset=['runtimeMinutes', 'genres'])

In [10]:
# Check value counts of 'titleType' column
basics['titleType'].value_counts()

short           278441
movie           203635
tvEpisode       173936
video           115674
tvSeries         36847
tvMovie          25959
tvSpecial         9295
tvMiniSeries      5900
tvShort           4100
videoGame          194
Name: titleType, dtype: int64

In [11]:
# Keep only titleType == movie
basics = basics[basics['titleType'] == "movie"]

In [12]:
# Convert the startYear column to float data type
basics['startYear'] = basics['startYear'].astype(float)

In [13]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203635 entries, 8 to 10064469
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          203635 non-null  object 
 1   titleType       203635 non-null  object 
 2   primaryTitle    203635 non-null  object 
 3   originalTitle   203635 non-null  object 
 4   isAdult         203635 non-null  object 
 5   startYear       200097 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  203635 non-null  object 
 8   genres          203635 non-null  object 
dtypes: float64(1), object(8)
memory usage: 15.5+ MB


In [14]:
# Filter dataframe using 'startYear' column
basics = basics.loc[(basics['startYear'] >= 2000.0)
                   & (basics['startYear'] <= 2021.0)]

In [15]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

## Final Dataframe Test

In [18]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34801,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61113,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67487,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67665,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86792,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [17]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81900 entries, 34801 to 10064469
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81900 non-null  object 
 1   titleType       81900 non-null  object 
 2   primaryTitle    81900 non-null  object 
 3   originalTitle   81900 non-null  object 
 4   isAdult         81900 non-null  object 
 5   startYear       81900 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  81900 non-null  object 
 8   genres          81900 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.2+ MB


In [19]:
# Save new dataframe to 'Data' folder.
basics.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)

### IMDB Data Sources:
- Downloads Page - https://developer.imdb.com/non-commercial-datasets/
- Data Dictionary - https://developer.imdb.com/non-commercial-datasets/
- IMDB Logo - https://www.themoviedb.org/about/logos-attribution

